# Definition of some basic support to import numpy arrays into VTK Objects.

Work in progress.

Written by Jean M Favre, Swiss National Supercomputing Center, as a proof of
concept. It is a very rudimentary thing. It needs generalization for CellData,
for vtkPolyData, for 1D and 2D grids and parallel support. It assumes 3D grids.

Tested on Piz Daint Sat 10 Oct 2020 01:15:44 PM CEST

In [ ]:
from paraview.simple import *
import numpy as np

In [ ]:
import vtk
from vtk.numpy_interface import algorithms as algs
from vtk.util import numpy_support as vtknp

class vtkStructuredGridFromArrays:
    def __init__(self):
        self.__mesh = vtk.vtkStructuredGrid()
        self.__dims = [-1,-1,-1]
    
    def SetVCoordinates(self, cArray):
        assert len(cArray.shape) == 1
        self.__mesh.Points = vtknp.numpy_to_vtk(cArray)
        self.__dims[0] = cArray.shape[0]
        
        self.__mesh.SetDimensions(self.__dims)
        
    def SetCoordinates(self, cArray0, cArray1, cArray2):
        assert len(cArray0.shape) == 3
        assert len(cArray1.shape) == 3
        assert len(cArray2.shape) == 3
        assert cArray2.shape == cArray1.shape
        assert cArray2.shape == cArray0.shape
        points = vtk.vtkPoints()
        self.__mesh.SetPoints(points)
        points.SetData(vtknp.numpy_to_vtk(
           algs.make_vector(cArray0.ravel(), cArray1.ravel(), cArray2.ravel()))
           )
        self.__dims = np.flip(cArray0.shape)
        self.__mesh.SetDimensions(self.__dims)
        
    def GetDimensions(self):
        self.__dims = self.__mesh.GetDimensions()
        return self.__dims
        
    def AddArray(self, dArray, name):
        assert np.prod(dArray.shape) == np.prod(self.__dims)
        vtkarr = vtknp.numpy_to_vtk(dArray.ravel())
        vtkarr.SetName(name)
        self.__mesh.GetPointData().AddArray(vtkarr)
        if len(dArray.shape) == 1:
          self.__mesh.GetPointData().SetActiveScalars(name)
    def GetOutput(self):
        return self.__mesh
        
class vtkRectGridFromArrays:
    def __init__(self):
        self.__mesh = vtk.vtkRectilinearGrid()
        self.__dims = [-1,-1,-1]

    def SetCoordinates(self, cArray0, cArray1, cArray2):
        assert len(cArray0.shape) == 1
        self.__mesh.SetXCoordinates(vtknp.numpy_to_vtk(cArray0))
        self.__dims[0] = cArray0.shape[0]
        
        assert len(cArray1.shape) == 1
        self.__mesh.SetYCoordinates(vtknp.numpy_to_vtk(cArray1))
        self.__dims[1] = cArray1.shape[0]

        assert len(cArray2.shape) == 1
        self.__mesh.SetZCoordinates(vtknp.numpy_to_vtk(cArray2))
        self.__dims[2] = cArray2.shape[0]
        
        self.__mesh.SetDimensions(self.__dims)

    def GetDimensions(self):
        return self.__mesh.GetDimensions()

    def AddArray(self, dArray, name):
        assert np.prod(dArray.shape) == np.prod(self.__dims)
        vtkarr = vtknp.numpy_to_vtk(dArray.ravel())
        vtkarr.SetName(name)
        self.__mesh.GetPointData().AddArray(vtkarr)
        self.__mesh.GetPointData().SetActiveScalars(name)

    def GetOutput(self):
        return self.__mesh

# a very rudimentary Point Cloud
class vtkPointSetFromArrays:
    def __init__(self):
        self.__mesh = vtk.vtkPolyData()
        self.__nnodes = 0
    
    def Make_PolyVertex(self):
      assert self.__nnodes > 0
      mlist = vtk.vtkIdList()
      mlist.SetNumberOfIds(self.__nnodes)
      for ii in range(self.__nnodes):
        mlist.SetId(ii, ii)
      self.__mesh.Allocate(1)
      self.__mesh.InsertNextCell(vtk.VTK_POLY_VERTEX, mlist)

    def SetVCoordinates(self, cArray):
        assert len(cArray.shape) == 2 and cArray.shape[1] == 3
        points = vtk.vtkPoints()
        self.__mesh.SetPoints(points)
        points.SetData(vtknp.numpy_to_vtk(cArray))
        self.__nnodes = cArray.shape[0]
        self.Make_PolyVertex()

    def SetCoordinates(self, cArray0, cArray1, cArray2):
        assert len(cArray0.shape) == 1
        assert len(cArray1.shape) == 1
        assert len(cArray2.shape) == 1
        assert cArray2.shape == cArray1.shape
        assert cArray2.shape == cArray0.shape
        points = vtk.vtkPoints()
        self.__mesh.SetPoints(points)
        points.SetData(vtknp.numpy_to_vtk(
          algs.make_vector(cArray0, cArray1, cArray2))
          )
        self.__nnodes = cArray0.shape[0]
        self.Make_PolyVertex()

    def AddArray(self, dArray, name):
        assert dArray.shape[0] == self.__nnodes
        vtkarr = vtknp.numpy_to_vtk(dArray.ravel())
        vtkarr.SetName(name)
        self.__mesh.GetPointData().AddArray(vtkarr)
        if len(dArray.shape) == 1:
          self.__mesh.GetPointData().SetActiveScalars(name)

    def GetOutput(self):
        return self.__mesh

In [ ]:
# Testing a regular cartesian grid, a.k.a. vtkImageData

In [ ]:
dims = [64,64,64]
bounds = (-10.0, 10.0, -10.0, 10.0,-10.0, 10.0)
xaxis = np.linspace(bounds[0], bounds[1], dims[0])
yaxis = np.linspace(bounds[2], bounds[3], dims[1])
zaxis = np.linspace(bounds[4], bounds[5], dims[2])
[xc,yc,zc] = np.meshgrid(zaxis,yaxis,xaxis, indexing="ij")
data = xc**2 + yc**2 + zc**2

from vtk.util.vtkImageImportFromArray import vtkImageImportFromArray
iD = vtkImageImportFromArray()
iD.SetArray(data) # it is given the name "scalars" by default
iD.SetDataOrigin((bounds[0], bounds[2], bounds[4]))
iD.SetDataSpacing(((bounds[1]-bounds[0])/(dims[0]-1),
                   (bounds[3]-bounds[2])/(dims[0]-1),
                   (bounds[5]-bounds[4])/(dims[0]-1)))
iD.Update()
iD.GetOutput().GetPointData().SetActiveScalars("scalars")

print("mesh dimensions = ", iD.GetOutput().GetDimensions())
assert iD.GetOutput().GetPointData().GetArray(0).GetName() == 'scalars'
datarange = iD.GetOutput().GetPointData().GetArray(0).GetRange()
print("data scalar range = ", datarange)

# create a new 'PVTrivialProducer'
pVTrivialProducer0 = PVTrivialProducer()
obj = pVTrivialProducer0.GetClientSideObject()
obj.SetOutput(iD.GetOutput())
pVTrivialProducer0.UpdatePipeline()

rep0 = Show(pVTrivialProducer0)
rep0.Representation = 'Surface With Edges'

In [ ]:
view = GetRenderView()
from ipyparaview.widgets import PVDisplay
disp = PVDisplay(view)
w = display(disp)

In [ ]:
rep0.Representation = 'Surface'
ColorBy(rep0, "scalars")

In [ ]:
Hide(pVTrivialProducer0)

# create a rectilinear grid

In [ ]:
dims = [64,64,64] # Radius dimension, Theta dimension, Z dimension
xaxis = np.linspace(0., 1., dims[0])
xaxis = xaxis**2
yaxis = np.linspace(0., 1., dims[1])
yaxis = np.sqrt(yaxis)
zaxis = np.linspace(0., 1., dims[2])

[xc,yc,zc] = np.meshgrid(zaxis,yaxis,xaxis, indexing="ij")
data = np.sqrt(xc**2 + yc**2 + zc**2)

rG = vtkRectGridFromArrays()
rG.SetCoordinates(xaxis, yaxis, zaxis)
rG.AddArray(data, "scalars")

print("RectilinearGrid dimensions = ", rG.GetDimensions())
assert rG.GetOutput().GetPointData().GetArray(0).GetName() == 'scalars'
datarange = rG.GetOutput().GetPointData().GetArray("scalars").GetRange()
print("RectilinearGrid  data scalar range = ", datarange)

# create a new 'PVTrivialProducer'
pVTrivialProducer1 = PVTrivialProducer()
obj = pVTrivialProducer1.GetClientSideObject()
obj.SetOutput(rG.GetOutput())
pVTrivialProducer1.UpdatePipeline()

rep1 = Show(pVTrivialProducer1)
rep1.Representation = 'Surface With Edges'

In [ ]:
ResetCamera()

In [ ]:
rep1.Representation = 'Surface'
ColorBy(rep1, 'scalars')

In [ ]:
Hide(pVTrivialProducer1)

# create a cylindrical - structured grid

In [ ]:
dims = [13,27,15] # Radius dimension, Theta dimension, Z dimension
Raxis = np.linspace(1., 2., dims[0])
Thetaaxis = np.linspace(0.,np.pi*1.5, dims[1])
Zaxis = np.linspace(0., 2.0, dims[2])
Z, t, r = np.meshgrid(Zaxis, Thetaaxis, Raxis, indexing="ij")
X = r * np.cos(t)
Y = r * np.sin(t)

NsG = vtkStructuredGridFromArrays()
NsG.SetCoordinates(X, Y, Z)
NsG.AddArray(r.ravel(), "radius")
NsG.AddArray(Z.ravel(), "z-elevation")
NsG.AddArray(t.ravel(), "angle")

print("StructuredGrid mesh dimensions = ", NsG.GetDimensions())
print("StructuredGrid mesh bounds = ", NsG.GetOutput().GetBounds())
assert NsG.GetOutput().GetPointData().GetArray(1).GetName() == 'z-elevation'
datarange = NsG.GetOutput().GetPointData().GetArray("z-elevation").GetRange()
print("StructuredGrid data scalar range = ", datarange)

# create a new 'PVTrivialProducer'
pVTrivialProducer2 = PVTrivialProducer()
obj = pVTrivialProducer2.GetClientSideObject()
obj.SetOutput(NsG.GetOutput())
pVTrivialProducer2.UpdatePipeline()

rep2 = Show(pVTrivialProducer2)
rep2.Representation = 'Surface With Edges'


In [ ]:
ColorBy(rep2, "radius")

In [ ]:
ResetCamera()

In [ ]:
Hide(pVTrivialProducer2)

# create a a point cloud - vtkPolyData

In [ ]:
nbpts = 10000
X = np.random.random_sample((nbpts, 3))

PC = vtkPointSetFromArrays()
#PC.SetCoordinates(X[:,0], X[:,1],X[:,2])
PC.SetVCoordinates(X)
PC.AddArray(X[:,0]*X[:,0] + X[:,1]*X[:,1] + X[:,2]*X[:,2], "distancesq")
PC.AddArray(X[:,0], "X")

print("Point Cloud has", PC.GetOutput().GetNumberOfPoints(), "vertices")
# create a new 'PVTrivialProducer'
pVTrivialProducer3 = PVTrivialProducer(guiName="Point Cloud")
obj = pVTrivialProducer3.GetClientSideObject()
obj.SetOutput(PC.GetOutput())
pVTrivialProducer3.UpdatePipeline()

rep3=Show(pVTrivialProducer3)
rep3.Representation = 'Points'

In [ ]:
# saving data is trivially done, just like in the GUI of ParaView.
# Using the GUI, you would save the Active Source,
# In batch mode, we can save any object in the visualization pipeline
# by using its proxyname

In [ ]:
SaveData("/users/jfavre/uniformGrid.vti",     proxy=pVTrivialProducer0)
SaveData("/users/jfavre/rectilinearGrid.vtr", proxy=pVTrivialProducer1)
SaveData("/users/jfavre/structuredGrid.vts",  proxy=pVTrivialProducer2)
SaveData("/users/jfavre/pointcloud.vtp",      proxy=pVTrivialProducer3)